# Feature Analysis
Feature analysis visualizers are designed to visualize instances in data space in order to detect features or targets that might impact downstream fitting. Because ML operates on high-dimensional data sets (usually several dozen!), the visualizers focus on aggregation, optimization, and other techniques to give overviews of the data. It is our intent that the steering process will allow the data scientist to zoom and filter and explore the relationships between their instances and between dimensions.

In [113]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from pandas.plotting import scatter_matrix

from yellowbrick.features import Rank2D
from yellowbrick.features import RadViz
from yellowbrick.features import ParallelCoordinates
from yellowbrick.features import FeatureImportances

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

### Load a dataset

I expect you to be able to fill write the code necessary to load your own dataset. It is not uncommon perform basic wrangling techniques after loading the data, such as dropping columns, changing datatypes, or renaming columns. Include that code here as well.

Lastly, it is very common that datasets are stored as CSV or Excel files, which can be stored in memory as a Pandas Dataframe. Pandas is an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for Python programmers and data scientists. Below is are two lines of code that load and print the first five rows of an Excel spreadsheet.

Example:
```python
df = pd.read_excel('filepath/file.xlsx')
df.head()
```

In [101]:
### Load your dataset.

## Correlations

### Jointplot

Like pairplots, a jointplot shows the distribution between features. It also shows individual distributions of the features being compared.

Example:
```python
sns.jointplot("fare", "age", df)
```

In [102]:
### Create a jointplot

### Scatter Plot Matrices

A scatter plot matrix is a grid of scatter plots, where each feature is plotted against every other feature. This helps you visualize the pairwise relationships between features. Use the Pandas `scatter_matrix()` function to create a scatter matrix from your Pandas Dataframe. For practice, start with a maximum of four features from your dataset. The argument `diagonal` refers to the visualization that populates the diagonal of the matrix. Since the diagonals represent a univariate analysis, it is common for scatter matrix functions to populate it with a histogram.

Example:
```python
subset = df[['feature1','featuer2', 'feature3', 'feature4']]

scatter_matrix(areas, alpha=0.2, diagonal='kde')
```

In [98]:
### Create a scatter matrix below.

### Pairplot

This is the scatter plot matrix from Pandas in Seaborn. Use the `.pairplot()` function to create a scatter matrix for four features in your dataset. In this case, you can pass a target to the `hue` argument.

Example:
```python
g = sns.pairplot(data=df[['feature1', 'feature2', 'feature3', 'feature4']], hue='target')
```

In [ ]:
### Write the code for a pairplot below.

### Rank 2D

A two-dimensional ranking of features utilizes a ranking algorithm that takes into account pairs of features at a time (e.g. joint plot analysis). The pairs of features are then ranked by score and visualized using the lower left triangle of a feature co-occurence matrix.

By default, the `Rank2D` visualizer utilizes the Pearson correlation score to detect colinear relationships.

Alternatively, we can utilize the covariance ranking algorithm, which attempts to compute the mean value of the product of deviations of variates from their respective means. Covariance loosely attempts to detect a colinear relationship between features. Try substituting `algorithm='covariance'` above to see the difference!

#### Define your features

Let's define the labels for our features that we will pass into the visualizer. Get a list of your features by calling the `.columns()` function on your Pandas Dataframe. If your target is included in the columns, then drop your target first before creating the features list.

Example:
```python
features = df.drop(columns=['target']).columns().tolist()
```

In [104]:
### Create a list of your features.

#### Create the visualization

First, we will instantiate the Rank2D visualizer, passing in the labels for our features. In one line, we fit the data by passing in our data. Then we will transform our data using the fitted parameters. Lastly, we will show the visualization.

Example:
```python
visualizer = Rank2D(algorithm='pearson', features=features, size=(1080, 720))
visualizer.fit(X, y)
visualizer.transform(X)
visualizer.show()
```

In [108]:
# Create the visualization.

## Separability

### Radviz (Classification Only)

`RadViz` is a multivariate data visualization algorithm that plots each feature dimension uniformly around the circumference of a circle then plots points on the interior of the circle such that the point normalizes its values on the axes from the center to each arc. This mechanism allows as many dimensions as will easily fit on a circle, greatly expanding the dimensionality of the visualization.


#### Define your classes (target)

In order for this visualization to work, you must pass a numeric representation of your target. For example, if your targets are binary, say, "unoccupid" and an "occupid" apartment, then those targets would need to be converted to "0" and "1" or "1" and "2".

Example:
```python
le = preprocessing.LabelEncoder()
le.fit(df['target'])
target = le.transform(df['class'])
```

In [ ]:
### Define a variable with your classes

#### Define your features

You must pass to the visualization the labels for your features. Get a list of your features by calling the `.columns()` function on your Pandas Dataframe. If your target is included in the columns, then drop your target first before creating the features list.

Example:
```python
features = df.drop(columns=['target']).columns().tolist()
```

In [ ]:
### Create a variable with your feature labels.

#### Create the visualization

First, we will instantiate the Radviz visualizer, passing in the labels for our classes, the features, and a size argument. The next two lines of data fit your data you the visualizer, transform your data on the fitted parameters, and finally showing the visualization.

Example:
```python
visualizer = RadViz(classes=le.classes_, features=features, size=(1080, 720))
visualizer.fit(df[features], target)
visualizer.transform(df[features])
visualizer.show()
```

Data scientists use this method to detect separability between classes. Is there an opportunity to learn from the feature set or is there just too much noise?

In [ ]:
# Create the visualization.

Now, let's re-create the visualization after reordering the features. To reorder columns in your Dataframe, pass a list of your columns in a different order than they already are in. For example, using `np.random.shuffle(features)` will shuffle the column labels.

Example:
```python
np.random.shuffle(features)
df = df[features]
```

In [ ]:
# Reorder your features in your dataset.

Using the same steps as above, re-create the visualization.

In [ ]:
# Recreate the visualization with reordered features.

### Parallel Coordinates (Classification Only)

Parallel coordinates is multi-dimensional feature visualization technique where the vertical axis is duplicated horizontally for each feature. Instances are displayed as a single line segment drawn from each vertical axes to the location representing their value for that feature. This allows many dimensions to be visualized at once; in fact given infinite horizontal space (e.g. a scrolling window), technically an infinite number of dimensions can be displayed!

Data scientists use this method to detect clusters of instances that have similar classes, and to note features that have high variance or different distributions.

By inspecting the visualization closely, we can see that the combination of transparency and overlap gives us the sense of groups of similar instances, sometimes referred to as "braids". If there are distinct braids of different classes, it suggests that there is enough separability that a classification algorithm might be able to discern between each class.

#### Create the visualization

Instantiate the ParallelCoordinate visualizer, passing in the labels for our classes and the features. The arguement `sample` in the example below is important, as it limits the amount of data visualized. The argument `shuffle` ensures that the selected data is drawn randomly. The first, transform, show, are similar to those you've already seen.

Example:
```python
visualizer = ParallelCoordinates(
    classes=classes, features=features, sample=0.05, 
    shuffle=True, size=(1080, 720)
)

visualizer.fit(X, y)
visualizer.transform(X)
visualizer.show()
```

In [116]:
### Create the visualization

However, the domain of each feature may make the visualization hard to interpret. To solve this problem, each feature should be scaled or normalized so they are approximately in the same domain.

Normalization techniques can be directly applied to the visualizer without pre-transforming the data (though you could also do this) by using the `normalize` parameter. Try using `minmax`, `minabs`, `standard`, `l1`, or `l2` normalization above to change perspectives!

Example:
```python
visualizer = ParallelCoordinates(
    classes=classes, features=features,
    normalize='standard', # This time we'll specify a normalizer
    sample=0.05, shuffle=True, size=(1080, 720)
)
visualizer.fit(X, y)
visualizer.transform(X)
visualizer.show()
```

In [109]:
### Recreate the visualization by transforming the data.

### Linear Regressions

Linear regressions are a popular method for assessing coorelation between two variables. For a tutorial see [here](https://stanford.edu/~mwaskom/software/seaborn/tutorial/regression.html).

Use the Seaborn `.regplot()` function to run a regression on two variables.

Example:
```python
sns.regplot(data=df, y='target', x='feature')
```

In [118]:
### Create a linear regression plot below.

### Feature Importances
The feature engineering process involves selecting the minimum required features to produce a valid model because the more features a model contains, the more complex it is (and the more sparse the data), therefore the more sensitive the model is to errors due to variance. A common approach to eliminating features is to describe their relative importance to a model, then eliminate weak features or combinations of features and re-evalute to see if the model fairs better during cross-validation.

Many model forms describe the underlying impact of features relative to each other. In scikit-learn, Decision Tree models and ensembles of trees such as Random Forest, Gradient Boosting, and Ada Boost provide a `feature_importances_` attribute when fitted. The Yellowbrick `FeatureImportances` visualizer utilizes this attribute to rank and plot relative importances.

#### Choose a machine learning model

To make things simple, we will begin with a Random Forest model. If you have a continuous target, use the Random Forest Regression. If you have a discrete target, then use the Random Forest Classifier.

Example:
```python
model = RandomForestClassifier()
```

In [111]:
### Instantiate a random forest model.

#### Create the visualization

To use the `FeatureImportance` visualization, pass in the model you've created and pass in the labels for your features. The next steps are to fit the data to the model with `.fit(X, y)` and then `.show()` to display the results.

With discrete targets, features are plotted against their *relative importance*, that is the percent importance of the most important feature. The visualizer also contains `features_` and `feature_importances_` attributes to get the ranked numeric values. For models that do not support a `feature_importances_` attribute, the `FeatureImportances` visualizer will also draw a bar plot for the `coef_` attribute that many linear models provide.

When using a model with a `coef_` attribute, it is better to set `relative=False` to draw the true magnitude of the coefficient (which may be negative). We can also specify our own set of labels if the dataset does not have column names or to print better titles.

Example:
```python
viz = FeatureImportances(model, labels=features, size=(1080, 720))
viz.fit(X, y)
viz.show()
```

In [115]:
### Create the feature importance visualization.